# IMPORTS

**Importing relevant modules in this space of all kinds**

In [0]:
!pip install ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6085 sha256=8c2a833f1f9f74fa49ba3abb46e0f7eb0f1251443e59d8700ac41c48c48c49a9
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


In [0]:
!pip install pescador

  Created wheel for pescador: filename=pescador-2.1.0-cp36-none-any.whl size=21076 sha256=d62f6c962108e9bc40fa878d509a7f78eddc7ccf1da9724bdf672b4b62869e1b
  Stored in directory: /root/.cache/pip/wheels/e7/e7/2d/de501314d1f03d8607aed0f6b48e50d8699f681745003aaa5b
Successfully built pescador


In [0]:
import torch

def check_memory():
    return torch.cuda.memory_allocated()

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#======================== Network based imports

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch import autograd
from torch import optim

#=========================Training related imports
import json
import numpy as np
import pprint
import pickle
import datetime

#========================== utility imports

import os
import time
import math
import torch
import random
import logging
import librosa
import argparse
import pescador
import ffmpeg
from matplotlib import pyplot as plt

In [0]:
cuda = True if torch.cuda.is_available() else False

# NETWORKS AND NORMS (ARCHITECTURE BASED CELLS)

**Transpose1d layer**  (The 8 minute video we saw. This is the convolution for audio type data)

In [0]:

class Transpose1dLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=11, upsample=None, output_padding=1):
        super(Transpose1dLayer, self).__init__()
        self.upsample = upsample

        self.upsample_layer = torch.nn.Upsample(scale_factor=upsample)
        reflection_pad = kernel_size // 2
        self.reflection_pad = nn.ConstantPad1d(reflection_pad, value=0)
        self.conv1d = torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride)
        self.Conv1dTrans = nn.ConvTranspose1d(in_channels, out_channels, kernel_size, stride, padding, output_padding)

    def forward(self, x):
        if self.upsample:
            return self.conv1d(self.reflection_pad(self.upsample_layer(x)))
        else:
            return self.Conv1dTrans(x)





class PhaseShuffle(nn.Module):
    """
    Performs phase shuffling, i.e. shifting feature axis of a 3D tensor
    by a random integer in {-n, n} and performing reflection padding where
    necessary.
    """
    # Copied from https://github.com/jtcramer/wavegan/blob/master/wavegan.py#L8
    def __init__(self, shift_factor):
        super(PhaseShuffle, self).__init__()
        self.shift_factor = shift_factor

    def forward(self, x):
        if self.shift_factor == 0:
            return x
        # uniform in (L, R)
        k_list = torch.Tensor(x.shape[0]).random_(0, 2 * self.shift_factor + 1) - self.shift_factor
        k_list = k_list.numpy().astype(int)

        # Combine sample indices into lists so that less shuffle operations
        # need to be performed
        k_map = {}
        for idx, k in enumerate(k_list):
            k = int(k)
            if k not in k_map:
                k_map[k] = []
            k_map[k].append(idx)

        # Make a copy of x for our output
        x_shuffle = x.clone()

        # Apply shuffle to each sample
        for k, idxs in k_map.items():
            if k > 0:
                x_shuffle[idxs] = F.pad(x[idxs][..., :-k], (k, 0), mode='reflect')
            else:
                x_shuffle[idxs] = F.pad(x[idxs][..., -k:], (0, -k), mode='reflect')

        assert x_shuffle.shape == x.shape, "{}, {}".format(x_shuffle.shape,
                                                       x.shape)
        return x_shuffle


class PhaseRemove(nn.Module):
    def __init__(self):
        super(PhaseRemove, self).__init__()

    def forward(self, x):
        pass



# """
# from torch.autograd import Variable
# x = Variable(torch.randn(10, 100))
# G = WaveGANGenerator(verbose=True, upsample=False)
# out = G(x)
# print(out.shape)
# D = WaveGANDiscriminator(verbose=True)
# out2 = D(out)
# print(out2.shape)
# """


***Generator*** class

In [0]:
class WaveGANGenerator(nn.Module):
    def __init__(self, model_size=64, ngpus=1, num_channels=1,
                 latent_dim=100, post_proc_filt_len=512,
                 verbose=False, upsample=True):
        super(WaveGANGenerator, self).__init__()
        self.ngpus = ngpus
        self.model_size = model_size  # d
        self.num_channels = num_channels  # c
        self.latent_di = latent_dim
        self.post_proc_filt_len = post_proc_filt_len
        self.verbose = verbose
        # "Dense" is the same meaning as fully connection.
        self.fc1 = nn.Linear(latent_dim, 256 * model_size)

        stride = 4
        if upsample:
            stride = 1
            upsample = 4
        self.deconv_1 = Transpose1dLayer(16 * model_size, 8 * model_size, 25, stride, upsample=upsample)
        self.deconv_2 = Transpose1dLayer(8 * model_size, 4 * model_size, 25, stride, upsample=upsample)
        self.deconv_3 = Transpose1dLayer(4 * model_size, 2 * model_size, 25, stride, upsample=upsample)
        self.deconv_4 = Transpose1dLayer(2 * model_size, model_size, 25, stride, upsample=upsample)
        self.deconv_5 = Transpose1dLayer(model_size, num_channels, 25, stride, upsample=upsample)

        if post_proc_filt_len:
            self.ppfilter1 = nn.Conv1d(num_channels, num_channels, post_proc_filt_len)

        for m in self.modules():
            if isinstance(m, nn.ConvTranspose1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)

    def forward(self, x):
        x = self.fc1(x).view(-1, 16 * self.model_size, 16)
        x = F.relu(x)
        if self.verbose:
            print(x.shape)

        x = F.relu(self.deconv_1(x))
        if self.verbose:
            print(x.shape)

        x = F.relu(self.deconv_2(x))
        if self.verbose:
            print(x.shape)

        x = F.relu(self.deconv_3(x))
        if self.verbose:
            print(x.shape)

        x = F.relu(self.deconv_4(x))
        if self.verbose:
            print(x.shape)

        output = torch.tanh(self.deconv_5(x))
        return output

***Discriminator*** Class

In [0]:
class WaveGANDiscriminator(nn.Module):
    def __init__(self, model_size=64, ngpus=1, num_channels=1, shift_factor=2,
                 alpha=0.2, verbose=False):
        super(WaveGANDiscriminator, self).__init__()
        self.model_size = model_size  # d
        self.ngpus = ngpus
        self.num_channels = num_channels  # c
        self.shift_factor = shift_factor  # n
        self.alpha = alpha
        self.verbose = verbose

        self.conv1 = nn.Conv1d(num_channels, model_size, 25, stride=4, padding=11)
        self.conv2 = nn.Conv1d(model_size, 2 * model_size, 25, stride=4, padding=11)
        self.conv3 = nn.Conv1d(2 * model_size, 4 * model_size, 25, stride=4, padding=11)
        self.conv4 = nn.Conv1d(4 * model_size, 8 * model_size, 25, stride=4, padding=11)
        self.conv5 = nn.Conv1d(8 * model_size, 16 * model_size, 25, stride=4, padding=11)

        self.ps1 = PhaseShuffle(shift_factor)
        self.ps2 = PhaseShuffle(shift_factor)
        self.ps3 = PhaseShuffle(shift_factor)
        self.ps4 = PhaseShuffle(shift_factor)

        self.fc1 = nn.Linear(256 * model_size, 1)

        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), negative_slope=self.alpha)
        if self.verbose:
            print(x.shape)
        x = self.ps1(x)

        x = F.leaky_relu(self.conv2(x), negative_slope=self.alpha)
        if self.verbose:
            print(x.shape)
        x = self.ps2(x)

        x = F.leaky_relu(self.conv3(x), negative_slope=self.alpha)
        if self.verbose:
            print(x.shape)
        x = self.ps3(x)

        x = F.leaky_relu(self.conv4(x), negative_slope=self.alpha)
        if self.verbose:
            print(x.shape)
        x = self.ps4(x)

        x = F.leaky_relu(self.conv5(x), negative_slope=self.alpha)
        if self.verbose:
            print(x.shape)

        x = x.view(-1, 256 * self.model_size)
        if self.verbose:
            print(x.shape)

        return self.fc1(x)

norms.py  for images. May need to change this according to audio 

In [0]:
def sobolev_norm(input, s=1, c=5):
    torch.cuda.empty_cache()
    signal_ndim = 2
    
    #fourier transform of input -- [real, imaginary]
    real_input = input
    #print(real_input.shape)
    imaginary_input = torch.zeros_like(input).cuda()
    fourier_transform = torch.fft(torch.stack((real_input, imaginary_input), -1),
                                  signal_ndim=signal_ndim)
        
    #compution the scale \xi
    #original N, M = fourier_transform.shape[2], fourier_transform.shape[2]

    N, M = fourier_transform.shape[1], fourier_transform.shape[1]
                                  
    ns = torch.arange(0, N).type(torch.FloatTensor).cuda() / N
    ms = torch.arange(0, M).type(torch.FloatTensor).cuda() / M
                                  
    xi_x, xi_y = torch.meshgrid([ms, ns])
    squared_xi = xi_x[None, None, :, :] ** 2 +\
                 xi_y[None, None, :, :] ** 2
    scaled_xi = (1 + c * squared_xi) ** (s * 0.5)

    #print(scaled_xi.shape, scaled_xi.shape)
    #print(fourier_transform.shape)
    #print(torch.stack([scaled_xi, scaled_xi], -1).shape)

    #the derivative in Sobolev norm is replaced by multiplication of \xi and fourier transform
    #print(fourier_transform.unsqueeze(0).unsqueeze(0).shape)


    # derivative = torch.stack([scaled_xi, scaled_xi], -1) * fourier_transform  #ORIGINAL CODE
    torch.cuda.empty_cache()
    
    
    #final inverse fourier transform
    # output = torch.ifft(derivative, signal_ndim=signal_ndim)    #ORIGINAL CODE

    output = torch.ifft((torch.stack([scaled_xi, scaled_xi], -1) * fourier_transform), signal_ndim=signal_ndim)  #modified code


    
    #we only need the real part as an answer
    output = output[..., 0]
    
    return output

def lp_norm(input, p=None):
    input = input.view(input.size(0), -1).type(torch.FloatTensor)
    
    #in order to find stable norm the normalization is performed
    #\|x\| = alpha * \|(x / alpha)\|
    #we will also try to avoid zero elements in alpha
    epsilon = 1e-5
    
    alpha, _ = torch.max((torch.abs(input) + epsilon), dim=1)
    output = alpha * torch.norm(input / alpha[:, None], p=p, dim=1)
    return output

# UTILITY FUNCTIONS

In [0]:
def init_console_logger(logger, verbose=False):
    stream_handler = logging.StreamHandler()
    if verbose:
        stream_handler.setLevel(logging.DEBUG)
    else:
        stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('[%(levelname)s] %(message)s')
    stream_handler.setFormatter(formatter)
    file_handler = logging.FileHandler("model.log")
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)

In [0]:
def make_path(output_path):
    if not os.path.isdir(output_path):
        os.makedirs(output_path)
    return output_path

def numpy_to_var(numpy_data, cuda):
    """
    Convert numpy array to Variable.
    """
    data = numpy_data[:, np.newaxis, :]
    data = torch.Tensor(data)
    if cuda:
        data = data.cuda()
    return Variable(data, requires_grad=False)


def plot_loss(D_cost_train, D_wass_train, D_cost_valid, D_wass_valid,
              G_cost, save_path):
    assert len(D_cost_train) == len(D_wass_train) == len(D_cost_valid) == len(D_wass_valid) == len(G_cost)

    save_path = os.path.join(save_path, "loss_curve.png")

    x = range(len(D_cost_train))

    y1 = D_cost_train
    y2 = D_wass_train
    y3 = D_cost_valid
    y4 = D_wass_valid
    y5 = G_cost

    plt.plot(x, y1, label='D_loss_train')
    plt.plot(x, y2, label='D_wass_train')
    plt.plot(x, y3, label='D_loss_valid')
    plt.plot(x, y4, label='D_wass_valid')
    plt.plot(x, y5, label='G_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    plt.savefig(save_path)


#This is gradient penalty function from the original code. 
# Please make sure to insert proper comments regarding which gradient penalty is being applied
#Considering that most of our project is based on that


l2_norm = 2
dual_norm = 1 / (1 - 1 / l2_norm) if l2_norm != 1 else np.inf

c_for_sobolev = 5.0
s_for_sobolev = 0

def calc_gradient_penalty(net_dis, real_data, fake_data, batch_size, lmbda, use_cuda=False):
    # Compute interpolation factors
    alpha = torch.rand(batch_size, 1, 1)
    alpha = alpha.expand(real_data.size())
    alpha = alpha.cuda() if use_cuda else alpha

    # Interpolate between real and fake data.
    interpolates = alpha * real_data + (1 - alpha) * fake_data
    if use_cuda:
        interpolates = interpolates.cuda()
    interpolates = autograd.Variable(interpolates, requires_grad=True)

    # Evaluate discriminator
    disc_interpolates = net_dis(interpolates)

    

    # Obtain gradients of the discriminator with respect to the inputs
    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).cuda() if use_cuda else
                              torch.ones(disc_interpolates.size()),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)

    #============== LAMBDA GAMMA
    dual_sobolev_gradients = sobolev_norm(gradients, s=-s_for_sobolev, c=c_for_sobolev)
    gradients_norm = lp_norm(dual_sobolev_gradients, p=dual_norm)

    lambda_ = lp_norm(sobolev_norm(real_data,  s=s_for_sobolev, c=c_for_sobolev),p=l2_norm).mean()
    
    
    gamma_ = lp_norm(sobolev_norm(real_data, s=-s_for_sobolev, c=c_for_sobolev),p=dual_norm).mean()

    prob_images = net_dis(real_data)

    #==============

    # Compute MSE between 1.0 and the gradient of the norm penalty to make discriminator
    # to be a 1-Lipschitz function.
    #gradient_penalty = lmbda * ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    torch.cuda.empty_cache()
    gradient_penalty = ((gradients_norm.float().cuda() / gamma_.float().cuda()  - 1) ** 2).mean() * lambda_.float().cuda()  + 1e-5 * (prob_images.float().cuda()  ** 2).mean()
    
    return gradient_penalty

In [0]:
def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def save_samples(epoch_samples, epoch, output_dir, fs=16000):
    """
    Save output samples.
    """
    sample_dir = make_path(os.path.join(output_dir, str(epoch)))

    for idx, sample in enumerate(epoch_samples):
        output_path = os.path.join(sample_dir, "{}.wav".format(idx+1))
        sample = sample[0]
        librosa.output.write_wav(output_path, sample, fs)


# Adapted from @jtcramer https://github.com/jtcramer/wavegan/blob/master/sample.py.  #og window length 16384
def sample_generator(filepath, window_length=16384, fs=16000):
    """
    Audio sample generator
    """
    try:
        audio_data, _ = librosa.load(filepath, sr=fs)

        # Clip magnitude
        max_mag = np.max(np.abs(audio_data))
        if max_mag > 1:
            audio_data /= max_mag
    except Exception as e:
        LOGGER.error("Could not load {}: {}".format(filepath, str(e)))
        raise StopIteration

    # Pad audio to >= window_length.
    audio_len = len(audio_data)
    if audio_len < window_length:
        pad_length = window_length - audio_len
        left_pad = pad_length // 2
        right_pad = pad_length - left_pad

        audio_data = np.pad(audio_data, (left_pad, right_pad), mode='constant')
        audio_len = len(audio_data)

    while True:
        if audio_len == window_length:
            # If we only have a single 1*window_length audio, just yield.
            sample = audio_data
        else:
            # Sample a random window from the audio
            start_idx = np.random.randint(0, (audio_len - window_length) // 2)
            end_idx = start_idx + window_length
            sample = audio_data[start_idx:end_idx]

        sample = sample.astype('float32')
        assert not np.any(np.isnan(sample))

        yield {'X': sample}


def get_all_audio_filepaths(audio_dir):
    return [os.path.join(root, fname)
            for (root, dir_names, file_names) in os.walk(audio_dir, followlinks=True)
            for fname in file_names
            if (fname.lower().endswith('.wav') or fname.lower().endswith('.mp3'))]


def batch_generator(audio_path_list, batch_size):
    streamers = []
    for audio_path in audio_path_list:
        s = pescador.Streamer(sample_generator, audio_path)
        streamers.append(s)

    mux = pescador.ShuffledMux(streamers)
    batch_gen = pescador.buffer_stream(mux, batch_size)

    return batch_gen


def split_data(audio_path_list, valid_ratio, test_ratio, batch_size):
    num_files = len(audio_path_list)
    num_valid = int(np.ceil(num_files * valid_ratio))
    num_test = int(np.ceil(num_files * test_ratio))
    num_train = num_files - num_valid - num_test

    if not (num_valid > 0 and num_test > 0 and num_train > 0):
        LOGGER.error("Please download DATASET '{}' and put it under current path !".format(DATASET_NAME))

    # Random shuffle the audio_path_list for splitting.
    random.shuffle(audio_path_list)

    valid_files = audio_path_list[:num_valid]
    test_files = audio_path_list[num_valid:num_valid + num_test]
    train_files = audio_path_list[num_valid + num_test:]
    train_size = len(train_files)

    train_data = batch_generator(train_files, batch_size)
    valid_data = batch_generator(valid_files, batch_size)
    test_data = batch_generator(test_files, batch_size)

    return train_data, valid_data, test_data, train_size

# Training

Code blocks from **train.py** starts here

In [0]:
# =============Logger===============
LOGGER = logging.getLogger('wavegan')
LOGGER.setLevel(logging.DEBUG)

LOGGER.info('Initialized logger.')
init_console_logger(LOGGER)

In [0]:
# =============Parameters===============
# args = parse_arguments()
# epochs = args['num_epochs']
# batch_size = args['batch_size']
# latent_dim = args['latent_dim']
# ngpus = args['ngpus']
# model_size = args['model_size']
# model_dir = make_path(os.path.join(args['output_dir'],
#                                    datetime.datetime.now().strftime("%Y%m%d%H%M%S")))
# args['model_dir'] = model_dir
# # save samples for every N epochs.
# epochs_per_sample = args['epochs_per_sample']
# # gradient penalty regularization factor.
# lmbda = args['lmbda']

In [0]:
#MANUALLY ADDING ARGUMENTS

model_size= 8                          #'Model size parameter used in WaveGAN' default=64,
shift_factor= 2                         #default=2, 'Maximum shift used by phase shuffle'
batch_shuffle=True
verbose=False
post_proc_filt_len=512                  #default=512 Length of post processing filter used by generator. Set to 0 to disable.'
alpha= 0.2                              #Slope of negative part of LReLU used by discriminator
valid_ratio=0.1                          #Ratio of audio files used for validation
test_ratio=0.1
batch_size= 1
epochs= 60
ngpus=1
latent_dim= 100                         #Size of latent dimension used by generator
epochs_per_sample=10                     #How many epochs between every set of samples generated for inspection
sample_size= 10                         # Generate 10 samples every sample generation.
lmbda=10.0                                  #Gradient penalty regularization factor default=10
learning_rate=1e-4
beta1=0.5
beta2=0.9
audio_dir='/content/drive/My Drive/WaveGANpy/piano'
output_dir= '/content/drive/My Drive/WaveGANpy/piano_outputBanachsize8/'  #<---------------Need to set whatever /content/file

DATASET_NAME='piano'

#=====just trying out stuff
#v=True
#psb=True


#==============================

#    parser.add_argument('-v', '--verbose', dest='verbose', action='store_true')   
# Had no clue what to do with --verbose, I suspect its something to do with logging

#parser.add_argument('-psb', '--phase-shuffle-batchwise', dest='batch_shuffle', action='store_true',help='If true, apply phase shuffle to entire batches rather than individual samples')




In [0]:
# Dir   #might also need to change the .pkl to .pth or whatever kind of network saving method we want to use
#Lets talk about .pkl, .pth , .etc choices whenever we come across this comment

#audio_dir = args['audio_dir']
#output_dir = args['output_dir']

netD_path = os.path.join(output_dir, "discriminator.pth")
netG_path = os.path.join(output_dir, "generator.pth")

Initializing Generator and Discriminator

In [0]:
# =============Network===============
netG = WaveGANGenerator(model_size=model_size, ngpus=ngpus, latent_dim=latent_dim, upsample=True)
netD = WaveGANDiscriminator(model_size=model_size, ngpus=ngpus)

if cuda:
    netG = torch.nn.DataParallel(netG).cuda()
    netD = torch.nn.DataParallel(netD).cuda()

In [0]:
#loading weights from saved models (optional)
#Dipam seriously need to introduce better checkpointing to this code. 
#Also need to check whether this thing is even loading actual weights or some bullshit that just restarts the process from zero
#Please do not forget to remove my weird comments 

# netG.load_state_dict(torch.load(netG_path))
# netD.load_state_dict(torch.load(netD_path))

# netG.load_state_dict(torch.load(netG_path)['model_state_dict'])
# netD.load_state_dict(torch.load(netD_path)['model_state_dict'])

In [0]:
#Initialization of Optimizers. 
#Need to check out what the hell this TTUR rule is. 


# "Two time-scale update rule"(TTUR) to update netD 4x faster than netG.


optimizerG = optim.Adam(netG.parameters(), lr=learning_rate, betas=(beta1, beta2))
optimizerD = optim.Adam(netD.parameters(), lr=learning_rate, betas=(beta1, beta2))

In [0]:
# Sample noise used for generated output.
sample_noise = torch.randn(sample_size, latent_dim)
if cuda:
    sample_noise = sample_noise.cuda()
sample_noise_Var = autograd.Variable(sample_noise, requires_grad=False)



In [0]:
check_memory()

4117504

In [0]:
# Save config.
# LOGGER.info('Saving configurations...')
# config_path = os.path.join(model_dir, 'config.json')
# with open(config_path, 'w') as f:
#     json.dump(args, f)

#THIS CODE CELL CONTAINS SOME args nonsense... need to remove it considering we have removed the argparse thing

In [0]:
!ls '/content/drive/My Drive/WaveGANpy/piano_outputst/'

discriminator_50.pth  generator_50.pth


Loading data  
* (will need to clean out args and configure it to read /drive/content)*

In [0]:
# Load data.
LOGGER.info('Loading audio data...')
audio_paths = get_all_audio_filepaths(audio_dir)
train_data, valid_data, test_data, train_size = split_data(audio_paths, valid_ratio, test_ratio, batch_size)
TOTAL_TRAIN_SAMPLES = train_size
BATCH_NUM = TOTAL_TRAIN_SAMPLES // batch_size
print(TOTAL_TRAIN_SAMPLES, BATCH_NUM)

train_iter = iter(train_data)
valid_iter = iter(valid_data)
test_iter = iter(test_data)

[INFO] Loading audio data...


15 15


In [0]:
import pdb

How to prevent Google Colab from disconnecting ?

function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)

Actual Training


In [0]:
from torch.autograd import Variable

In [0]:
check_memory()


4117504

In [0]:
# =============Train===============
history = []
D_costs_train = []
D_wasses_train = []
D_costs_valid = []
D_wasses_valid = []
G_costs = []

start = time.time()
LOGGER.info('Starting training...EPOCHS={}, BATCH_SIZE={}, BATCH_NUM={}'.format(epochs, batch_size, BATCH_NUM))
for epoch in range(1, epochs+1):
    # print("Entered")
    LOGGER.info("{} Epoch: {}/{}".format(time_since(start), epoch, epochs))

    D_cost_train_epoch = []
    D_wass_train_epoch = []
    D_cost_valid_epoch = []
    D_wass_valid_epoch = []
    G_cost_epoch = []
    # pdb.set_trace()
    for i in range(1, BATCH_NUM+1):
        # print("Second")
        # Set Discriminator parameters to require gradients.
        for p in netD.parameters():
            p.requires_grad = True

        one = torch.tensor(1, dtype=torch.float)
        neg_one = one * -1
        if cuda:
            one = one.cuda()
            neg_one = neg_one.cuda()
        #############################
        # (1) Train Discriminator
        #############################
        for iter_dis in range(5):
            netD.zero_grad()

            # Noise
            noise = torch.Tensor(batch_size, latent_dim).uniform_(-1, 1)
            if cuda:
                noise = noise.cuda()
            noise_Var = Variable(noise, requires_grad=False)

            real_data_Var = numpy_to_var(next(train_iter)['X'], cuda)

            # a) compute loss contribution from real training data
            D_real = netD(real_data_Var)
            D_real = D_real.mean()  # avg loss
            D_real.backward(neg_one)  # loss * -1

            # b) compute loss contribution from generated data, then backprop.
            fake = autograd.Variable(netG(noise_Var).data)
            D_fake = netD(fake)
            D_fake = D_fake.mean()
            D_fake.backward(one)

            # c) compute gradient penalty and backprop
            gradient_penalty = calc_gradient_penalty(netD, real_data_Var.data,
                                                     fake.data, batch_size, lmbda,
                                                     use_cuda=cuda)
            gradient_penalty.backward(one)

            # Compute cost * Wassertein loss..
            D_cost_train = D_fake - D_real + gradient_penalty
            D_wass_train = D_real - D_fake

            # Update gradient of discriminator.
            optimizerD.step()

            #############################
            # (2) Compute Valid data
            #############################
            netD.zero_grad()

            valid_data_Var = numpy_to_var(next(valid_iter)['X'], cuda)
            D_real_valid = netD(valid_data_Var)
            D_real_valid = D_real_valid.mean()  # avg loss

            # b) compute loss contribution from generated data, then backprop.
            fake_valid = netG(noise_Var)
            D_fake_valid = netD(fake_valid)
            D_fake_valid = D_fake_valid.mean()

            # c) compute gradient penalty and backprop
            gradient_penalty_valid = calc_gradient_penalty(netD, valid_data_Var.data,
                                                           fake_valid.data, batch_size, lmbda,
                                                           use_cuda=cuda)
            # Compute metrics and record in batch history.
            D_cost_valid = D_fake_valid - D_real_valid + gradient_penalty_valid
            D_wass_valid = D_real_valid - D_fake_valid

            if cuda:
                D_cost_train = D_cost_train.cpu()
                D_wass_train = D_wass_train.cpu()
                D_cost_valid = D_cost_valid.cpu()
                D_wass_valid = D_wass_valid.cpu()

            # Record costs
            D_cost_train_epoch.append(D_cost_train.data.numpy())
            D_wass_train_epoch.append(D_wass_train.data.numpy())
            D_cost_valid_epoch.append(D_cost_valid.data.numpy())
            D_wass_valid_epoch.append(D_wass_valid.data.numpy())

        #############################
        # (3) Train Generator
        #############################
        # Prevent discriminator update.
        for p in netD.parameters():
            p.requires_grad = False

        # Reset generator gradients
        netG.zero_grad()

        # Noise
        noise = torch.Tensor(batch_size, latent_dim).uniform_(-1, 1)
        if cuda:
            noise = noise.cuda()
        noise_Var = Variable(noise, requires_grad=False)

        fake = netG(noise_Var)
        G = netD(fake)
        G = G.mean()

        # Update gradients.
        G.backward(neg_one)
        G_cost = -G

        optimizerG.step()

        # Record costs
        if cuda:
            G_cost = G_cost.cpu()
        G_cost_epoch.append(G_cost.data.numpy())

        if i % (BATCH_NUM // 5) == 0:
            LOGGER.info("{} Epoch={} Batch: {}/{} D_c:{:.4f} | D_w:{:.4f} | G:{:.4f}".format(time_since(start), epoch,
                                                                                             i, BATCH_NUM,
                                                                                             D_cost_train.data.numpy(),
                                                                                             D_wass_train.data.numpy(),
                                                                                             G_cost.data.numpy()))
                                                                                             
                                                                             

    # Save the average cost of batches in every epoch.
    D_cost_train_epoch_avg = sum(D_cost_train_epoch) / float(len(D_cost_train_epoch))
    D_wass_train_epoch_avg = sum(D_wass_train_epoch) / float(len(D_wass_train_epoch))
    D_cost_valid_epoch_avg = sum(D_cost_valid_epoch) / float(len(D_cost_valid_epoch))
    D_wass_valid_epoch_avg = sum(D_wass_valid_epoch) / float(len(D_wass_valid_epoch))
    G_cost_epoch_avg = sum(G_cost_epoch) / float(len(G_cost_epoch))

    D_costs_train.append(D_cost_train_epoch_avg)
    D_wasses_train.append(D_wass_train_epoch_avg)
    D_costs_valid.append(D_cost_valid_epoch_avg)
    D_wasses_valid.append(D_wass_valid_epoch_avg)
    G_costs.append(G_cost_epoch_avg)

    LOGGER.info("{} D_cost_train:{:.4f} | D_wass_train:{:.4f} | D_cost_valid:{:.4f} | D_wass_valid:{:.4f} | "
                "G_cost:{:.4f}".format(time_since(start),
                                       D_cost_train_epoch_avg,
                                       D_wass_train_epoch_avg,
                                       D_cost_valid_epoch_avg,
                                       D_wass_valid_epoch_avg,
                                       G_cost_epoch_avg))

    # Generate audio samples.
    if epoch % epochs_per_sample == 0:
        LOGGER.info("Generating samples...")
        sample_out = netG(sample_noise_Var)
        if cuda:
            sample_out = sample_out.cpu()
        sample_out = sample_out.data.numpy()
        save_samples(sample_out, epoch, output_dir)
        LOGGER.info("Saving models...")
        netD_path = os.path.join(output_dir, "discriminator.pth")
        netG_path = os.path.join(output_dir, "generator.pth")
        torch.save({
            'epoch': epoch,
            'model_state_dict': netD.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict()
            
            
            }, netD_path)
        torch.save({
            'epoch': epoch,
            'model_state_dict': netG.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict()}, netG_path)

        
        
     

    # TODO
    # Early stopping by Inception Score(IS)

LOGGER.info('>>>>>>>Training finished !<<<<<<<')

[INFO] Starting training...EPOCHS=60, BATCH_SIZE=1, BATCH_NUM=15
[INFO] 0m 0s Epoch: 1/60
[INFO] 2m 53s Epoch=1 Batch: 3/15 D_c:1.2737 | D_w:3.6486 | G:0.1720
[INFO] 5m 8s Epoch=1 Batch: 6/15 D_c:-13.8438 | D_w:13.8533 | G:-0.5596
[INFO] 7m 7s Epoch=1 Batch: 9/15 D_c:7.7469 | D_w:-7.5185 | G:-23.6588
[INFO] 9m 2s Epoch=1 Batch: 12/15 D_c:12.8275 | D_w:-11.7037 | G:-13.5089
[INFO] 10m 56s Epoch=1 Batch: 15/15 D_c:-24.8375 | D_w:27.9325 | G:39.3182
[INFO] 10m 56s D_cost_train:2.4720 | D_wass_train:0.2573 | D_cost_valid:0.1980 | D_wass_valid:1.6306 | G_cost:-3.3814
[INFO] 10m 56s Epoch: 2/60
[INFO] 12m 51s Epoch=2 Batch: 3/15 D_c:-26.0861 | D_w:26.3356 | G:55.4477
[INFO] 14m 45s Epoch=2 Batch: 6/15 D_c:16.5906 | D_w:-15.4196 | G:9.7281
[INFO] 16m 40s Epoch=2 Batch: 9/15 D_c:4.2979 | D_w:-0.2684 | G:6.9392
[INFO] 18m 34s Epoch=2 Batch: 12/15 D_c:-28.9577 | D_w:29.0271 | G:16.5062
[INFO] 20m 29s Epoch=2 Batch: 15/15 D_c:-68.6876 | D_w:75.0842 | G:31.0052
[INFO] 20m 29s D_cost_train:-13.2635

Plotting loss and saving the final model

In [0]:
plot_loss(D_costs_train, D_wasses_train,D_costs_valid, D_wasses_valid, G_costs, output_dir)

torch.save(netD.state_dict(), netD_path)
torch.save(netG.state_dict(), netG_path)